In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llama-3.1/transformers/8b-instruct/1/model.safetensors.index.json
/kaggle/input/llama-3.1/transformers/8b-instruct/1/model-00003-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/1/config.json
/kaggle/input/llama-3.1/transformers/8b-instruct/1/model-00001-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/1/README.md
/kaggle/input/llama-3.1/transformers/8b-instruct/1/USE_POLICY.md
/kaggle/input/llama-3.1/transformers/8b-instruct/1/tokenizer.json
/kaggle/input/llama-3.1/transformers/8b-instruct/1/tokenizer_config.json
/kaggle/input/llama-3.1/transformers/8b-instruct/1/model-00004-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/1/special_tokens_map.json
/kaggle/input/llama-3.1/transformers/8b-instruct/1/model-00002-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/1/generation_config.json
/kaggle/input/shafi-vai-with-testing-data/testing_data.csv
/kaggle/input/shafi-vai-with-testing-data/RawDat

In [ ]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U accelerate
%pip install -U peft
%pip install -U trl
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U accelerate
%pip install -U trl
%pip install -U peft
%pip install -U wandb

In [5]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split

In [6]:
df = pd.read_csv("/kaggle/input/shafi-vai-with-testing-data/RawData.csv")
df

,comment,Sentiment
0,এমন ভালবাসা ইসলাম সমর্থন করে না বিয়ের আগে কোন...,Hate Speech
1,নোংরা মনমানসিকতার একটা লিমিট আছে এখানে এই প...,Hate Speech
2,হারি আপ ব্রাদার অ্যাটেনশন please believe me ...,Hate Speech
3,এ নাটকের সবথেকে বাজে দিক হচ্ছে এখানে বাবা মা...,Hate Speech
4,এটা অনেক কষ্টের আসলে মেয়েরা অনেক স্বার্থপর ...,Hate Speech
...,...,...
2315,বসের নাটকের জন্য শুধু অপেহ্মায় থাকি\n,Normal Speech
2316,ভাই নাইস নাইস তিশা আপুকে আরো নাইস \n,Normal Speech
2317,আজকের খুশির দিনেও কাঁদিয়ে দিলি রে ভাই কোলকাতা...,Normal Speech
2318,অসাধারন নিশো ভাই আপনি আসলেই সেরা\n,Normal Speech


In [7]:
df1=pd.read_csv("/kaggle/input/shafi-vai-with-testing-data/testing_data.csv")
df1

,comment,Sentiment
0,বছরের সেরা নাটক বলতে হবে আপনারা কি বলেন...,Normal Speech
1,রকম মাইয়া কেও ভালোবাসতো আমারে লালা জান দিয়া...,Normal Speech
2,নাটকটা তেমন ভালো লাগেনাইঅপ্রয়োজনীয় কথা বার্ত...,Hate Speech
3,নিশো আর তিশা কি নাটক জগতটা পচাবে নাকি এদের ...,Hate Speech
4,হালারপুতে থাকেই ট্রেন্ড টপিক নিয়া যখন কিছু থ...,Hate Speech
...,...,...
253,সালার পুত্র মুশফিক কুকুরের বাচ্চা,Hate Speech
254,সালি একটা মাল ফুটকি মারতে ভালো লাগবে,Hate Speech
255,সৃজিতকে কুত্তা দিয়ে চোদালে বুঝতে পারবে ভার্জিন...,Hate Speech
256,হালার দেখে পুরাই মাথাই নস্ট\n,Hate Speech


In [8]:
unique_sentiments = df['Sentiment'].unique()
print(unique_sentiments)

['Hate Speech' 'Normal Speech']


In [9]:
# Shuffle the DataFrame and select only 3000 rows
df = df.sample(frac=1, random_state=85)

# Split the DataFrame
train_size = 0.8
eval_size = 0.1

# Calculate sizes
train_end = int(train_size * len(df))
eval_end = train_end + int(eval_size * len(df))

# Split the data
X_train = df[:train_end]
X_eval = df[train_end:eval_end]
X_test = df1

# Define the prompt generation functions
def generate_prompt(data_point):
    return f"""
            Classify the text into Hate Speech, Normal Speech and return the answer as the corresponding Sentiment label.
text: {data_point["comment"]}
label: {data_point["Sentiment"]}""".strip()

def generate_test_prompt(data_point):
    return f"""
            Classify the text into Hate Speech, Normal Speech  and return the answer as the corresponding Sentiment label.
text: {data_point["comment"]}
label: """.strip("Sentiment")

# Generate prompts for training and evaluation data
X_train.loc[:,'comment'] = X_train.apply(generate_prompt, axis=1)
X_eval.loc[:,'comment'] = X_eval.apply(generate_prompt, axis=1)

# Generate test prompts and extract true labels
y_true = X_test.loc[:,'Sentiment']
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["comment"])

In [10]:
X_train.Sentiment.value_counts()

Sentiment
Hate Speech      1174
Normal Speech     682
Name: count, dtype: int64

In [11]:
# Convert to datasets
train_data = Dataset.from_pandas(X_train[["comment"]])
eval_data = Dataset.from_pandas(X_eval[["comment"]])

In [12]:
base_model_name = "/kaggle/input/llama-3.1/transformers/8b-instruct/1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype="float16",
    quantization_config=bnb_config, 
)

model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [13]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id

In [14]:
def predict(test, model, tokenizer):
    y_pred = []
    categories = ["Hate Speech", "Normal Speech"]
    
    for i in tqdm(range(len(test))):
        prompt = test.iloc[i]["comment"]
        pipe = pipeline(task="text-generation", 
                        model=model, 
                        tokenizer=tokenizer, 
                        max_new_tokens=2, 
                        temperature=0.1)
        
        result = pipe(prompt)
        answer = result[0]['generated_text'].split("label:")[-1].strip()
        
        # Determine the predicted category
        for category in categories:
            if category.lower() in answer.lower():
                y_pred.append(category)
                break
        else:
            y_pred.append("none")
    
    return y_pred

In [15]:
y_pred = predict(X_test, model, tokenizer)

100%|██████████| 258/258 [01:27<00:00,  2.94it/s]


In [16]:
def evaluate(y_true, y_pred):
    labels = ["Hate Speech", "Normal Speech"]
    mapping = {label: idx for idx, label in enumerate(labels)}
    
    def map_func(x):
        return mapping.get(x, -1)  # Map to -1 if not found, but should not occur with correct data
    
    y_true_mapped = np.vectorize(map_func)(y_true)
    y_pred_mapped = np.vectorize(map_func)(y_pred)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true_mapped, y_pred=y_pred_mapped)
    print(f'Accuracy: {accuracy:.3f}')
    
    # Generate accuracy report
    unique_labels = set(y_true_mapped)  # Get unique labels
    
    for label in unique_labels:
        label_indices = [i for i in range(len(y_true_mapped)) if y_true_mapped[i] == label]
        label_y_true = [y_true_mapped[i] for i in label_indices]
        label_y_pred = [y_pred_mapped[i] for i in label_indices]
        label_accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {labels[label]}: {label_accuracy:.3f}')
        
    # Generate classification report
    class_report = classification_report(y_true=y_true_mapped, y_pred=y_pred_mapped, target_names=labels, labels=list(range(len(labels))))
    print('\nClassification Report:')
    print(class_report)
    
    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true_mapped, y_pred=y_pred_mapped, labels=list(range(len(labels))))
    print('\nConfusion Matrix:')
    print(conf_matrix)

In [21]:
evaluate(y_true, y_pred)

Accuracy: 0.364
Accuracy for label Hate Speech: 0.038
Accuracy for label Normal Speech: 0.871

Classification Report:
               precision    recall  f1-score   support

  Hate Speech       0.60      0.04      0.07       157
Normal Speech       0.80      0.87      0.83       101

    micro avg       0.78      0.36      0.50       258
    macro avg       0.70      0.45      0.45       258
 weighted avg       0.68      0.36      0.37       258


Confusion Matrix:
[[ 6 22]
 [ 4 88]]


In [17]:
!pip install shap

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [18]:
import shap
import torch

In [19]:
import bitsandbytes as bnb

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:  # needed for 16 bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)
modules = find_all_linear_names(model)
modules
output_dir="llama-3.1-shafi-vai"

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules,
)

training_arguments = TrainingArguments(
    output_dir=output_dir,                    
    num_train_epochs=3,                      
    per_device_train_batch_size=1,           
    gradient_accumulation_steps=8,           
    gradient_checkpointing=True,              
    optim="paged_adamw_32bit",
    logging_steps=1,                         
    learning_rate=2e-4,                       
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,                        
    max_steps=-1,
    warmup_ratio=0.03,                      
    group_by_length=False,
    lr_scheduler_type="cosine",            
    report_to="wandb",                  
    eval_strategy="steps",              
    eval_steps = 0.2
)

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="comment",
    tokenizer=tokenizer,
    max_seq_length=512,
    packing=False,
    dataset_kwargs={
    "add_special_tokens": False,
    "append_concat_token": False,
    }
)

Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length, dataset_kwargs. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
You passed a `dataset_kwargs` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.


Map:   0%|          | 0/1856 [00:00<?, ? examples/s]

Map:   0%|          | 0/232 [00:00<?, ? examples/s]

`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.


In [20]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
`torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.


Step,Training Loss,Validation Loss
140,0.491300,0.764379
280,0.478300,0.736136
420,0.632900,0.716322
560,0.419300,0.739342


torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
`torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.


TrainOutput(global_step=696, training_loss=0.5946709797128864, metrics={'train_runtime': 5707.9444, 'train_samples_per_second': 0.975, 'train_steps_per_second': 0.122, 'total_flos': 2.543503603426099e+16, 'train_loss': 0.5946709797128864, 'epoch': 3.0})

In [23]:
import wandb
wandb.finish()
model.config.use_cache = True

eval/loss,█▄▁▄
eval/runtime,█▁▂▃
eval/samples_per_second,▁█▇▆
eval/steps_per_second,▁█▆▅
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,██▃▃▂▃▁▂▂▃▁▂▃▁▂▂▂▂▁▂▁▁▂▂▂▃▂▃▂▄▂▃▂▂▃▂▂▂▃▁
train/learning_rate,▄███████▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
train/loss,█▃▃▃▂▂▂▃▃▃▂▃▂▂▁▃▂▂▂▂▂▂▂▃▂▂▂▂▂▁▂▂▂▂▁▂▁▁▂▁
eval/loss,0.73934
eval/runtime,62.7054


In [24]:
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

('llama-3.1-shafi-vai/tokenizer_config.json',
 'llama-3.1-shafi-vai/special_tokens_map.json',
 'llama-3.1-shafi-vai/tokenizer.json')

In [25]:
y_pred = predict(X_test, model, tokenizer)
evaluate(y_true, y_pred)

100%|██████████| 258/258 [02:00<00:00,  2.13it/s]

Accuracy: 0.992
Accuracy for label Hate Speech: 1.000
Accuracy for label Normal Speech: 0.980

Classification Report:
               precision    recall  f1-score   support

  Hate Speech       0.99      1.00      0.99       157
Normal Speech       1.00      0.98      0.99       101

     accuracy                           0.99       258
    macro avg       0.99      0.99      0.99       258
 weighted avg       0.99      0.99      0.99       258


Confusion Matrix:
[[157   0]
 [  2  99]]
